In [1]:
import pandas as pd
import numpy as np

data_raw = pd.read_csv('../raw/지역사회건강조사 1차 전처리_2015.csv')

In [2]:
data_raw

,Unnamed: 0,만나이,성별,주관적 건강수준,격렬한 신체활동 일수,격렬한 신체활동 시간(시),격렬한 신체활동 시간(분),중등도 신체활동 일수,중등도 신체활동 시간(시),중등도 신체활동 시간(분),걷기 실천 일수,걷기 실천 시간(시),걷기 실천 시간(분),체중조절 경험 여부,주관적 스트레스 수준,우울감 경험 여부,행복감 지수,사회환경_동네 자연환경,사회환경_동네 생활환경,자치구
0,0,58,2,2,0,88,88,4,3,0,2,1,0,1,3,2,7,1,2,강남구
1,1,60,1,2,0,88,88,0,88,88,5,1,0,4,4,2,8,1,2,강남구
2,2,52,2,3,0,88,88,0,88,88,3,0,20,4,3,2,7,1,1,강남구
3,3,22,2,2,0,88,88,0,88,88,7,1,0,1,3,2,9,1,2,강남구
4,4,51,2,3,0,88,88,0,88,88,3,0,50,2,3,2,7,1,1,강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22999,22999,40,1,2,1,2,0,0,88,88,7,2,0,1,4,2,6,1,1,중랑구
23000,23000,26,2,3,2,1,0,2,0,30,4,0,20,3,2,2,5,1,1,중랑구
23001,23001,25,1,2,2,2,0,2,2,0,7,12,0,1,3,2,10,1,1,중랑구
23002,23002,27,2,1,1,2,0,0,88,88,5,0,40,1,2,2,7,2,1,중랑구


In [3]:
data_pbhlth_code = pd.read_excel('../raw/지역사회건강조사 2019년 원시자료 이용지침서 등/지역사회건강조사 2019 원시자료 참고사항_보건소정보, 세대유형.xlsx',header=3)
data_pbhlth_code = data_pbhlth_code[data_pbhlth_code['시도코드'] == 11][['보건소코드','보건소명']]
data_pbhlth_code["보건소명"] = data_pbhlth_code["보건소명"].str.replace('보건소', '', regex=True)
data_pbhlth_code.columns=['보건소코드','자치구']
list_region = list(data_pbhlth_code.to_dict()['자치구'].values()) + ['서울시']
list_region.reverse()

In [4]:
df = data_raw[['만나이','성별','자치구', '격렬한 신체활동 일수', '격렬한 신체활동 시간(시)','격렬한 신체활동 시간(분)']]

In [5]:
df1=df[df['격렬한 신체활동 일수'] != 99]
df2=df1[df1['격렬한 신체활동 일수'] != 77]
df3=df2[df2['격렬한 신체활동 시간(시)'] != 99]
df4=df3[df3['격렬한 신체활동 시간(시)'] != 77]

In [6]:
def transtime(df_activity) :
    if 0 <= df_activity['격렬한 신체활동 시간(시)'] <= 24 and df_activity['격렬한 신체활동 시간(분)'] <= 59 :
        return df_activity['격렬한 신체활동 시간(시)']*60 + df_activity['격렬한 신체활동 시간(분)']
    else :
        return 0
    
def health_index(df_activity) :
    if df_activity['격렬한 신체활동 일수'] < 3 :
        return 0
    elif 3 <= df_activity['격렬한 신체활동 일수'] and df_activity['격렬 시분 환산'] >= 20 :
        return 1
    else :
        return 0

In [7]:
df_activity = df4.copy() # 오류제거
df_activity["격렬 시분 환산"] = df_activity.apply(transtime, axis =1)
df_activity['격렬한 신체활동 실천'] = df_activity.apply(health_index, axis =1)
df_activity = df_activity.drop(['격렬한 신체활동 시간(시)','격렬한 신체활동 시간(분)','격렬한 신체활동 일수','격렬 시분 환산'],axis=1)

In [8]:
list_df = []
for value in list_region:
    temp = df_activity['자치구'] == value
    list_df.append(df_activity[temp])
list_df[0] = df_activity

In [9]:
list_age = ['19~200','19~44','45~64','65~74','75~200']
list_sex = ['계','남','여']
idx_sex = []
idx_age = []
idx_region = []
value_act = []
for i, value1 in enumerate(list_region):
    for j in list_age:
        for k in list_sex:            
            temp = list_df[i][list_df[i]['만나이']>=int(j.split('~')[0])]
            temp1 = temp[temp['만나이'] <= int(j.split('~')[1])]
            if k == '남':
                temp2 = temp1[temp1['성별'] == 1]
            elif k == '여':
                temp2 = temp1[temp1['성별'] == 2]
            else:
                temp2 = temp1
            temp3 = temp2['격렬한 신체활동 실천'].value_counts(normalize = True).to_dict()
            if 1 not in temp3:
                temp3[1] = 0
            if 0 not in temp3:
                temp3[0] = 0
            value_act.append(round(temp3[1]*100,1))
            
            if j == '19~200':
                idx_age.append('계')
            elif j == '75~200':
                idx_age.append('75 이상')
            else:
                idx_age.append(j)
            idx_sex.append(k)
            idx_region.append(value1)

In [10]:
df_final = pd.DataFrame({'격렬한 신체활동 실천':value_act, '지역':idx_region,'연령별':idx_age,'성별':idx_sex})

df_final = df_final.set_index(['지역','연령별','성별'])

df_final

격렬한 신체활동 실천
지역  연령별   성별             
서울시 계     계          15.8
          남          19.8
          여          12.7
    19~44 계          17.8
          남          21.8
...                   ...
강남구 65~74 남           7.3
          여           0.0
    75 이상 계          10.4
          남           9.1
          여          11.5

[390 rows x 1 columns]

In [11]:
df_final.to_csv("../raw/지역사회건강조사 (전처리된 데이터)_2015/지역사회건강조사 (격렬한 신체활동)_2015.csv", mode='w',encoding = 'utf-8-sig')